# import package

In [134]:
import re
import csv
import numpy as np
import pandas as pd

# data extract

In [135]:
#read_data
csv_files = pd.read_csv("/Users/wangzhuoxin/Desktop/NZ_Admin_JOBS.csv")
csv_files.head()

# change head name 
#df_new = df.rename(columns={'A': 'a'}, index={'ONE': 'one'})
csv_new = csv_files.rename(columns = {"字段1": "role","字段1_link": "website_link", "字段2":"company", "字段3":"location","字段4": 'post_day',"字段5":"classification"})
csv_new.head()

,role,website_link,company,location,post_day,classification
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: Taur...,"Featured,at",classification: Administration & Office Suppor...
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office Suppor...
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: Invercargill...,"1h ago,at",classification: Administration & Office Suppor...
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,location: CanterburyCanterburyarea: Christchur...,"4d ago,at,Private Advertiser",classification: Administration & Office Suppor...


# data transforming

## check data types

In [136]:
csv_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 6 columns):
role              2708 non-null object
website_link      2708 non-null object
company           2686 non-null object
location          2708 non-null object
post_day          2708 non-null object
classification    2708 non-null object
dtypes: object(6)
memory usage: 127.1+ KB


## change location column

### split location into location and area split location into location and area

In [137]:
#the argument expand = True means generate a data frame
loc_df = csv_new['location'].str.split(": ",expand=True)
loc_df.columns = ['delete','area','loc']
loc_df.head(2)

,delete,area,loc
0,location,Bay of PlentyBay of Plentyarea,TaurangaTauranga
1,location,Bay of PlentyBay of Plentyarea,TaurangaTauranga


### combine two dataframe

In [138]:
# combine two dataframe by using contact method, note:axis = 1 means row
combine_df = pd.concat([csv_new, loc_df], axis=1)
#print(combine_df.head(1))
combine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 9 columns):
role              2708 non-null object
website_link      2708 non-null object
company           2686 non-null object
location          2708 non-null object
post_day          2708 non-null object
classification    2708 non-null object
delete            2708 non-null object
area              2708 non-null object
loc               2167 non-null object
dtypes: object(9)
memory usage: 190.5+ KB


### drop location column called delete

In [139]:
# drop location column inplace = Flase,return a copy.Otherweise.
combine_df.drop(columns = ['delete','location'] ,inplace = True)

combine_df =combine_df.rename(columns = {'loc':'location'})
combine_df.head(2)


,role,website_link,company,post_day,classification,area,location
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,"Featured,at,Private Advertiser",classification: Administration & Office Suppor...,Bay of PlentyBay of Plentyarea,TaurangaTauranga
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,"Featured,at",classification: Administration & Office Suppor...,Bay of PlentyBay of Plentyarea,TaurangaTauranga


In [140]:
combine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 7 columns):
role              2708 non-null object
website_link      2708 non-null object
company           2686 non-null object
post_day          2708 non-null object
classification    2708 non-null object
area              2708 non-null object
location          2167 non-null object
dtypes: object(7)
memory usage: 148.2+ KB


## create salary column

### add new column called salary

In [141]:
print("the 24th row for location and salary information : {}".format(combine_df['location'][23])) 

the 24th row for location and salary information : Rodney & North ShoreRodney & North Shore,$19 to $22 per hour


In [142]:
# get salary infomation
salary_df = combine_df['location'].str.split(',',expand = True)
salary_df.tail()

,0,1,2,3
2703,Auckland CentralAuckland Central,None,None,None
2704,Wellington CentralWellington Central,Competitive hourly rate,None,None
2705,HamiltonHamilton,Competitive hourly rates $$,None,None
2706,HamiltonHamilton,Competitive hourly rates $$,None,None
2707,None,None,None,None


In [143]:
#give column0 and coulumn1 name
salary_df.columns = ['location1','salary','none1','none2']
salary_df.tail(1)

,location1,salary,none1,none2
2707,None,None,None,None


In [144]:
# combine two df
combine_df2 = pd.concat([combine_df,salary_df[['location1','salary']]],axis = 1 )
combine_df2.iloc[23,:]

role                                        Reception/Administrator
website_link      https://www.seek.co.nz/job/50556340?type=promo...
company                           Fishing and Leisure Group Limited
post_day                                                Featured,at
classification                                  $19 to $22 per hour
area                                           AucklandAucklandarea
location          Rodney & North ShoreRodney & North Shore,$19 t...
location1                  Rodney & North ShoreRodney & North Shore
salary                                          $19 to $22 per hour
Name: 23, dtype: object

In [145]:
#drop original location
combine_df2.drop(columns = 'location',inplace = True)
combine_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 8 columns):
role              2708 non-null object
website_link      2708 non-null object
company           2686 non-null object
post_day          2708 non-null object
classification    2708 non-null object
area              2708 non-null object
location1         2167 non-null object
salary            527 non-null object
dtypes: object(8)
memory usage: 169.4+ KB


In [146]:
# rename location1
combine_df2 = combine_df2.rename(columns = {'location1':'location'})
combine_df2.iloc[23,:]

role                                        Reception/Administrator
website_link      https://www.seek.co.nz/job/50556340?type=promo...
company                           Fishing and Leisure Group Limited
post_day                                                Featured,at
classification                                  $19 to $22 per hour
area                                           AucklandAucklandarea
location                   Rodney & North ShoreRodney & North Shore
salary                                          $19 to $22 per hour
Name: 23, dtype: object

## change classification column

In [147]:
# extract calssification:
combine_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 8 columns):
role              2708 non-null object
website_link      2708 non-null object
company           2686 non-null object
post_day          2708 non-null object
classification    2708 non-null object
area              2708 non-null object
location          2167 non-null object
salary            527 non-null object
dtypes: object(8)
memory usage: 169.4+ KB


In [148]:
classi =combine_df2['classification'].str.extract(r'(^classification.*)',expand = False)
# check classification whether have salary inside
classi.tail(3)

2705                                                  NaN
2706                                                  NaN
2707    classification: Administration & Office Suppor...
Name: classification, dtype: object

In [149]:
combine_df2['classi']= classi
combine_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 9 columns):
role              2708 non-null object
website_link      2708 non-null object
company           2686 non-null object
post_day          2708 non-null object
classification    2708 non-null object
area              2708 non-null object
location          2167 non-null object
salary            527 non-null object
classi            2105 non-null object
dtypes: object(9)
memory usage: 190.5+ KB


In [150]:
#drop classification column
combine_df2.drop(columns = ['classification'], inplace = True)
combine_df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 8 columns):
role            2708 non-null object
website_link    2708 non-null object
company         2686 non-null object
post_day        2708 non-null object
area            2708 non-null object
location        2167 non-null object
salary          527 non-null object
classi          2105 non-null object
dtypes: object(8)
memory usage: 169.4+ KB


In [151]:
#rename for combine_df
#data_trans_df = combine_df2.rename(columns = {'classi':'classification'})
#data_trans_df.info()

In [152]:
combine_df2.tail()

,role,website_link,company,post_day,area,location,salary,classi
2703,Key Account Manager,https://www.seek.co.nz/job/50490062?type=stand...,Hays Talent Solutions,"27d ago,at",AucklandAucklandarea,Auckland CentralAuckland Central,None,classification: Administration & Office Suppor...
2704,Executive Assistant,https://www.seek.co.nz/job/50488000?type=stand...,one eighty recruitment,"27d ago,at",WellingtonWellingtonarea,Wellington CentralWellington Central,Competitive hourly rate,NaN
2705,Temporary Office Roles,https://www.seek.co.nz/job/50524865?type=stand...,Asset Recruitment Ltd,"20d ago,at",WaikatoWaikatoarea,HamiltonHamilton,Competitive hourly rates $$,NaN
2706,Temporary Office Roles,https://www.seek.co.nz/job/50477118?type=stand...,Asset Recruitment Ltd,"28d ago,at",WaikatoWaikatoarea,HamiltonHamilton,Competitive hourly rates $$,NaN
2707,Executive Assistant,https://www.seek.co.nz/job/50496571?type=stand...,Alpha Recruitment - NZ,"26d ago,at",WellingtonWellington,None,None,classification: Administration & Office Suppor...


## split classi column into 2 pieces

In [153]:
#post day, area, location,classification

In [154]:
#classificatio

In [155]:
merge_df = combine_df2['classi'].str.split(': ',expand = True)
merge_df.iloc[1,1]

'Administration & Office SupportAdministration & Office SupportsubClassification'

In [156]:
merge_df.drop(columns = 0,inplace = True)

In [157]:
merge_df.columns = ['delete','cla']

In [158]:
merge_df.head(1)

,delete,cla
0,Administration & Office SupportAdministration ...,Office ManagementOffice Management


In [159]:
merge_sub_df = merge_df['delete'].str.split('&',expand = True)
merge_sub_df.columns = ['classification1','classification2','classification3']
merge_sub_df.head()


,classification1,classification2,classification3
0,Administration,Office SupportAdministration,Office SupportsubClassification
1,Administration,Office SupportAdministration,Office SupportsubClassification
2,Administration,Office SupportAdministration,Office SupportsubClassification
3,Administration,Office SupportAdministration,Office SupportsubClassification
4,Administration,Office SupportAdministration,Office SupportsubClassification


In [160]:
#merge these two sub dataframe
comb_df = pd.concat([merge_sub_df,merge_df],axis =1 )
comb_df.head(1)


,classification1,classification2,classification3,delete,cla
0,Administration,Office SupportAdministration,Office SupportsubClassification,Administration & Office SupportAdministration ...,Office ManagementOffice Management


In [161]:
comb_df.drop(columns ='delete',inplace = True)
comb_df = comb_df.rename(columns = {'cla':'classification4'})
comb_df.head(3)

,classification1,classification2,classification3,classification4
0,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management
1,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists
2,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther


In [162]:
combine_df3 = pd.concat([combine_df2, comb_df], axis=1)
combine_df3.head(4)

,role,website_link,company,post_day,area,location,salary,classi,classification1,classification2,classification3,classification4
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,"Featured,at,Private Advertiser",Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,classification: Administration & Office Suppor...,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,"Featured,at",Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,classification: Administration & Office Suppor...,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,"4d ago,at",AucklandAuckland,None,None,classification: Administration & Office Suppor...,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,"1h ago,at",SouthlandSouthlandarea,InvercargillInvercargill,None,classification: Administration & Office Suppor...,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...


In [163]:
#drop classi column

combine_df3.drop(columns = 'classi',inplace = True)
combine_df3.head(5)

,role,website_link,company,post_day,area,location,salary,classification1,classification2,classification3,classification4
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,"Featured,at,Private Advertiser",Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,"Featured,at",Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,"4d ago,at",AucklandAuckland,None,None,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,"1h ago,at",SouthlandSouthlandarea,InvercargillInvercargill,None,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,"4d ago,at,Private Advertiser",CanterburyCanterburyarea,ChristchurchChristchurch,None,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...


## change post_day column

In [164]:
post_df = combine_df3['post_day'].str.split(',at', expand = True)
post_df.columns = ['post_time','featured at']
post_df.head()

,post_time,featured at
0,Featured,",Private Advertiser"
1,Featured,
2,4d ago,
3,1h ago,
4,4d ago,",Private Advertiser"


In [165]:
combine_df4 = pd.concat([combine_df3,post_df],axis = 1)
combine_df4.head()

,role,website_link,company,post_day,area,location,salary,classification1,classification2,classification3,classification4,post_time,featured at
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,"Featured,at,Private Advertiser",Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management,Featured,",Private Advertiser"
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,"Featured,at",Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists,Featured,
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,"4d ago,at",AucklandAuckland,None,None,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther,4d ago,
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,"1h ago,at",SouthlandSouthlandarea,InvercargillInvercargill,None,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...,1h ago,
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,"4d ago,at,Private Advertiser",CanterburyCanterburyarea,ChristchurchChristchurch,None,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...,4d ago,",Private Advertiser"


In [166]:
#drop post_day columns
combine_df4.drop(columns = ['post_day'],inplace = True)
combine_df4.head()

,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,post_time,featured at
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management,Featured,",Private Advertiser"
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists,Featured,
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,None,None,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther,4d ago,
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea,InvercargillInvercargill,None,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...,1h ago,
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,CanterburyCanterburyarea,ChristchurchChristchurch,None,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...,4d ago,",Private Advertiser"


In [167]:
b =combine_df4['post_time'].str.extract(r'(\d*d)',expand = False)
combine_df4['post_time_1'] = b
combine_df4.head()

,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,post_time,featured at,post_time_1
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management,Featured,",Private Advertiser",d
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists,Featured,,d
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,None,None,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther,4d ago,,4d
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea,InvercargillInvercargill,None,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...,1h ago,,NaN
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,CanterburyCanterburyarea,ChristchurchChristchurch,None,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...,4d ago,",Private Advertiser",4d


In [168]:
po_time_df = combine_df4['post_time_1'].str.split('d',expand = True)
po_time_df.head(10)

,0,1
0,,
1,,
2,4,
3,NaN,NaN
4,4,
5,NaN,NaN
6,5,
7,NaN,NaN
8,NaN,NaN
9,3,


In [169]:
po_time_df = po_time_df.rename(columns = {0:'posted_time',1:"delete"})
po_time_df.drop(columns = 'delete', inplace= True)
po_time_df.head(10)

,posted_time
0,
1,
2,4
3,NaN
4,4
5,NaN
6,5
7,NaN
8,NaN
9,3


In [170]:
po_time_df =po_time_df.fillna(0)
po_time_df.head()

,posted_time
0,
1,
2,4
3,0
4,4


In [171]:

def post_time(a):
    if a == '' :
        return "Unknown"
    else:
        return a 
po_time_df['posted_time_1'] = po_time_df['posted_time'].apply(post_time)
po_time_df.head()





,posted_time,posted_time_1
0,,Unknown
1,,Unknown
2,4,4
3,0,0
4,4,4


In [172]:
def post_time2(a):
    if a == 'Unknown' :
        return 'Unknown'
    elif a == 0:
        return 0
    else:
        
        str1 = "-" + str(a) 
        return str1
        
    
po_time_df['posted_time_1'] = po_time_df['posted_time_1'].apply(post_time2)
po_time_df.head()


,posted_time,posted_time_1
0,,Unknown
1,,Unknown
2,4,-4
3,0,0
4,4,-4


In [173]:
#combine_df = pd.concat([csv_new, loc_df], axis=1)
combine_df5 = pd.concat([combine_df4,po_time_df],axis = 1 )
#combine_df5.head()
#drop post_time, post_time1 ,posted_time_1
combine_df5.drop(columns = ['post_time','post_time_1','posted_time'],inplace = True)
combine_df5.head()
#rename posted_time_1
combine_df5 = combine_df5.rename(columns = {'posted_time_1':'post_time'})
combine_df5.head()


,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,featured at,post_time
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management,",Private Advertiser",Unknown
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists,,Unknown
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,None,None,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther,,-4
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea,InvercargillInvercargill,None,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...,,0
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,CanterburyCanterburyarea,ChristchurchChristchurch,None,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...,",Private Advertiser",-4


## change featured at column,drop comma

In [174]:
fea_df = combine_df5['featured at'].str.split(',',expand = True)
fea_df.drop(columns = 0,inplace = True)
fea_df.columns = ['feature_at']
fea_df.head()

,feature_at
0,Private Advertiser
1,None
2,None
3,None
4,Private Advertiser


In [175]:
# combine to original dataframe than drop the featured at column

In [176]:
combine_df6 = pd.concat([combine_df5,fea_df],axis = 1)
#combine_df6.head()
combine_df6.drop(columns = 'featured at',inplace = True)

In [177]:
def feature(b):
    if b == None:
        return "Unknown"
    else:
        return b

In [178]:
combine_df6['feature_at']= combine_df6['feature_at'].apply(feature)

In [179]:
combine_df6.head()

,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,post_time,feature_at
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,NaN,Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management,Unknown,Private Advertiser
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea,TaurangaTauranga,None,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists,Unknown,Unknown
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,None,None,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther,-4,Unknown
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea,InvercargillInvercargill,None,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...,0,Unknown
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,NaN,CanterburyCanterburyarea,ChristchurchChristchurch,None,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...,-4,Private Advertiser


# data clean

## check missing value 

In [180]:
# we find company has missing value
combine_df6.isna().any()

role               False
website_link       False
company             True
area               False
location            True
salary              True
classification1     True
classification2     True
classification3     True
classification4     True
post_time          False
feature_at         False
dtype: bool

In [181]:
#fill company NA value
#nba["College"].fillna("No College", inplace = True) 
combine_df6['company'].fillna('Unknown',inplace = True)
#fill location NA value
combine_df6['location'].fillna('Unknown',inplace = True)
#fill salary NA value
combine_df6['salary'].fillna('Unknown',inplace = True)
#fill classification value
combine_df6['classification1'].fillna('Unknown',inplace = True)
combine_df6['classification2'].fillna('Unknown',inplace = True)
combine_df6['classification3'].fillna('Unknown',inplace = True)
combine_df6['classification4'].fillna('Unknown',inplace = True)
#fill feature_at value
combine_df6['feature_at'].fillna('Unknown',inplace = True)

combine_df6.head()

,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,post_time,feature_at
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Unknown,Bay of PlentyBay of Plentyarea,TaurangaTauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management,Unknown,Private Advertiser
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea,TaurangaTauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists,Unknown,Unknown
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,Unknown,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther,-4,Unknown
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea,InvercargillInvercargill,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...,0,Unknown
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Unknown,CanterburyCanterburyarea,ChristchurchChristchurch,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...,-4,Private Advertiser


In [182]:
#check whether have missing values
combine_df6.isna().any()

role               False
website_link       False
company            False
area               False
location           False
salary             False
classification1    False
classification2    False
classification3    False
classification4    False
post_time          False
feature_at         False
dtype: bool

## clean dulplicate value

### clean area column

In [183]:
# clean area column 
#firstly, drop area column
area_df = combine_df6['area'].str.split('area',expand = True)
area_df.drop(columns = 1, inplace = True)
area_df.columns = ['area1']



In [184]:
## combine area_df to the original datadrame
combine_df7 = pd.concat([combine_df6, area_df],axis = 1)
combine_df7.head()

,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,post_time,feature_at,area1
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Unknown,Bay of PlentyBay of Plentyarea,TaurangaTauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management,Unknown,Private Advertiser,Bay of PlentyBay of Plenty
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of PlentyBay of Plentyarea,TaurangaTauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists,Unknown,Unknown,Bay of PlentyBay of Plenty
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,AucklandAuckland,Unknown,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther,-4,Unknown,AucklandAuckland
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea,InvercargillInvercargill,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...,0,Unknown,SouthlandSouthland
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Unknown,CanterburyCanterburyarea,ChristchurchChristchurch,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...,-4,Private Advertiser,CanterburyCanterbury


In [185]:
def dulplicate_lines(a):
    
    if a == 'Unknown':
        return "Unknown"
    else:
        return a[len(a)//2:]

In [186]:
combine_df7['area'] = combine_df7['area1'].apply(dulplicate_lines)

In [187]:
combine_df7.drop(columns = 'area1',inplace = True)


In [188]:
combine_df7.head()

,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,post_time,feature_at
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Unknown,Bay of Plenty,TaurangaTauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management,Unknown,Private Advertiser
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of Plenty,TaurangaTauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists,Unknown,Unknown
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,Auckland,Unknown,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther,-4,Unknown
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,Southland,InvercargillInvercargill,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...,0,Unknown
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Unknown,Canterbury,ChristchurchChristchurch,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...,-4,Private Advertiser


### clean location column

In [189]:
combine_df7['location'] = combine_df7['location'].apply(dulplicate_lines)
combine_df7.head()

,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,post_time,feature_at
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Unknown,Bay of Plenty,Tauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Office ManagementOffice Management,Unknown,Private Advertiser
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of Plenty,Tauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,ReceptionistsReceptionists,Unknown,Unknown
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,Auckland,Unknown,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,OtherOther,-4,Unknown
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,Southland,Invercargill,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Administrative AssistantsAdministrative Assist...,0,Unknown
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Unknown,Canterbury,Christchurch,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Client & Sales AdministrationClient & Sales Ad...,-4,Private Advertiser


### clean classification4 column

In [202]:
combine_df7['classification4'] = combine_df7['classification4'].apply(dulplicate_lines)
combine_df7.head()

,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,post_time,feature_at,salary1
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Unknown,Bay of Plenty,Tauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,anagement,Unknown,Private Advertiser,Private Advertiser
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of Plenty,Tauranga,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,ionists,Unknown,Unknown,Unknown
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,Auckland,Unknown,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,her,-4,Unknown,Unknown
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,Southland,Invercargill,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,ve Assistants,0,Unknown,Unknown
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Unknown,Canterbury,Christchurch,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Administration,-4,Private Advertiser,Private Advertiser


### clean salary column

In [203]:
def salary1(a,b):
    if a == "Unknown" and b != "Unknown":
        return b
    elif a != "Unknown" and b== "Unknown":
        return a 
    else:
        return "Unknown"

In [204]:
combine_df7['salary1'] =combine_df7.apply(lambda x: salary1(x['salary'], x['feature_at']), axis=1)

In [207]:
combine_df7.tail()

,role,website_link,company,area,location,salary,classification1,classification2,classification3,classification4,post_time,feature_at,salary1
2703,Key Account Manager,https://www.seek.co.nz/job/50490062?type=stand...,Hays Talent Solutions,Auckland,Auckland Central,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,Administration,-27,Unknown,Unknown
2704,Executive Assistant,https://www.seek.co.nz/job/50488000?type=stand...,one eighty recruitment,Wellington,Wellington Central,Competitive hourly rate,Unknown,Unknown,Unknown,Unknown,-27,Unknown,Competitive hourly rate
2705,Temporary Office Roles,https://www.seek.co.nz/job/50524865?type=stand...,Asset Recruitment Ltd,Waikato,Hamilton,Competitive hourly rates $$,Unknown,Unknown,Unknown,Unknown,-20,Unknown,Competitive hourly rates $$
2706,Temporary Office Roles,https://www.seek.co.nz/job/50477118?type=stand...,Asset Recruitment Ltd,Waikato,Hamilton,Competitive hourly rates $$,Unknown,Unknown,Unknown,Unknown,-28,Unknown,Competitive hourly rates $$
2707,Executive Assistant,https://www.seek.co.nz/job/50496571?type=stand...,Alpha Recruitment - NZ,Wellington,Unknown,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,ecretarial,-26,Unknown,Unknown


In [210]:
combine_df7.drop(columns = 'salary',inplace = True)

In [212]:
combine_df7 = combine_df7.rename(columns = {"salary1":'salary'})

In [213]:
combine_df7.head(3)

,role,website_link,company,area,location,classification1,classification2,classification3,classification4,post_time,feature_at,salary
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Unknown,Bay of Plenty,Tauranga,Administration,Office SupportAdministration,Office SupportsubClassification,anagement,Unknown,Private Advertiser,Private Advertiser
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of Plenty,Tauranga,Administration,Office SupportAdministration,Office SupportsubClassification,ionists,Unknown,Unknown,Unknown
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,Auckland,Unknown,Administration,Office SupportAdministration,Office SupportsubClassification,her,-4,Unknown,Unknown
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,Southland,Invercargill,Administration,Office SupportAdministration,Office SupportsubClassification,ve Assistants,0,Unknown,Unknown
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Unknown,Canterbury,Christchurch,Administration,Office SupportAdministration,Office SupportsubClassification,Administration,-4,Private Advertiser,Private Advertiser
